# True randomness: demo of randquantum module 

This notebook gives a demonstration of the useful functions which yield *true* random numbers produced live from an experiment in quantum mechanics.  They are contained in the **randquantum** Python module under the **randomsys** project: https://git.io/randomsys 

Reliable and unbiased random numbers are needed for a range of applications from numerical modeling to cryptographic communications. While there are algorithms that can generate pseudo random numbers, they can never be perfectly random nor indeterministic. ANU researchers are generating *true random numbers from a physical quantum source* by splitting a beam of light into two beams and then measuring the power in each beam. Because light is quantised, the light intensity in each beam fluctuates about the mean. Those fluctuations, due ultimately to the quantum vacuum, can be converted into a source of random numbers. 

The rate at which the live bits are streamed is limited by the bandwidth of your internet connection. Every number is randomly generated in real time and cannot be predicted beforehand. Most pseudo-random numbers have a finite period after which the sequence repeats. The output herein will not have such periodicity.

Details and references can be found in the Appendix.

*CHANGE LOG*

    2015-10-05  First draft.

In [1]:
#  NOTEBOOK v4 settings and system details:      [00-tpl v4.15.0812]

#  Assume that the backend is LINUX (e.g. Ubuntu running bash shell):
print '\n ::  TIMESTAMP of last notebook execution:'
!date
print ' ::  IPython version:'
!ipython --version

#  Automatically RELOAD modified modules:
%load_ext autoreload
%autoreload 2
#           0 disables autoreload.

#  DISPLAY options
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  MATH display, use %%latex, rather than the following:
#                from IPython.display import Math
#                from IPython.display import Latex
#  Generate PLOTS inside notebook:
%matplotlib inline

print ' ::  Working directory (set as $workd):'
workd, = !pwd
print workd + '\n'


 ::  TIMESTAMP of last notebook execution:
Tue Oct  6 10:16:19 PDT 2015
 ::  IPython version:
3.2.1
 ::  Working directory (set as $workd):
/home/yaya/Dropbox/share/git/nous/randomsys/quantum



In [2]:
import randquantum as rq
import numpy as np   #  numpy is not a dependency, just here for stats.

## Truly random seed 

Most pseudo-random number generators require a initial starting point called a *seed*. It should obviously be random, and is usually derived from the computer current state such as time in milliseconds. The randquantum module by design does not need any seed -- in fact, it is a reliable source to **generate a random integer of any arbitrary length** (constrained only by memory size), suitable as a seed for other projects.

In [3]:
#  Demo random integer of length 76:
print rq.seed( 76 )

1511358160521810890058797362640387260100378360832593987897984201386416925859


The implication is that the outputs from randquantum are not replicable! Try it out in this IPython notebook (download and execute locally).

## Random digits and statistical testing

Here one call to rq.integer() will yield a single random digit:

In [4]:
rq.nine()

2

Let's now informally see the distribution across ten digits 0 through 9. It should look uniform, otherwise there is bias and this entire project becomes invalid. The formal tests for our *true* random numbers can be found at: http://qrng.anu.edu.au/NIST.php -- daily testing results are posted there.

In [5]:
counter = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range( 10000 ):
    d = rq.nine()
    counter[d] += 1
    
counter    

[991, 1006, 919, 1036, 1037, 1005, 1035, 956, 1035, 980]

Generating ten thousand samples, the count for each digit is roughly one thousand, so informally we can say, "not bad." The implicit point is that *you can place a randquantum **generator** function anywhere in your source code to create true random conditions indefintely.*

The online source of randomness supplies us with a list of *finite* length, but we overcome this API limitation by writing generators. The online dependency is somewhat problematic for offline projects so we have a simulated fallback, just in case (and warnings to stderr will be issued).

## Arbitrarily large random integers and permutations

Even for small len(x), the total number of permutations of x is 
larger than the period of most PSEUDO random number generators; 
this implies that most permutations of a long sequence can 
never be generated. But this is not true for randquantum_authentic;
though if the listing is huge, this could take a long time to finish.

In [6]:
#  Truly find a random integer between 0 and some arbitrary integer:
rq.randint( 98765432109876543210 )
#  L is for internal representation of long integer.

36535999703968882812L

Given the function **randint** we can pick out random element(s) from any arbitrarily large list, with or without replacement.

In [7]:
zoo = [ 'dog', 'cat', 'mice', 'rat', 'lion', 'tiger', 'snake']

#  Pick out two randomly:
rq.randpick( zoo, count=2, replace=False )

['rat', 'snake']

In [8]:
#  Now the difficult part in theory: repeatedly 
#  produce a true permutation of an arbitrarily large list:
rq.shuffle( zoo )

['dog', 'rat', 'lion', 'snake', 'tiger', 'cat', 'mice']

If every atom in the universe has a distinct label at a moment in time, and we could successfully shuffle those labels repeatedly since our generators do not have periodicity.

## Randomness from a Gaussian distribution

Having a true uniform distribution permits one to mathematically create a Gaussian distribution. That happens in the generator function **gauss**. We use the Kinderman Ratio of Uniform Deviates method.

In [10]:
#  Sample from Normal N(0, 1) distribution, five times:
for i in range( 5 ):
    print rq.gauss()

-1.02190523877
0.333454530226
0.704678891125
-0.603181418823
-0.260031274773


Let informally see if we are getting random samples from a N(0, 1). We the numpy module to compute the mean and standard deviation for us.

In [11]:
#  Insert a list into a numpy array:
gauss_output = np.array( rq.gaussquantum( 10000, mean= 0, sdev=1.0))

#  Module code shows gaussquantum returns a list which is used by gauss().

In [12]:
#  Verify sample size:
np.shape( gauss_output )

(10000,)

In [13]:
#  numpy calculates the sample mean:
gauss_output.mean()

-0.020067968419944671

In [14]:
#  numpy standard deviation method:
gauss_output.std()

1.0021490435024947

## Quintessential uniform real from [0, 1)

The sample values are truly random, but they are not from a continous semi-open line. FAQ: Why not?

Answer:  Well, these true random numbers originate from the **quantum** world where fundamentally everything is *discrete*. (Pseudo generators have a similar problem, but due to the limited numerical precision of floats which is hardware dependent.)

In [16]:
#  Sample from real interval [0, 1) uniformly:
for i in range( 5 ):
    print rq.real()

0.313694972149
0.682291905082
0.190966659035
0.51822690165
0.0209811551077


## Even more essential: boolean()

Getting a truly random binary output is at the heart of our experiments. From this alone, we can in theory simulate the evolution of the entire universe. Let there be photons!

In [17]:
#  Random boolean values:
for i in range( 5 ):
    print rq.boolean()

0
1
1
0
1


In [18]:
#  Use it freely in conditionals:
if rq.boolean():
    print "Lucky day!"
else:
    print "No pizza."

No pizza.


## Fun ready-made generators

The randquantum module allows you to create your own custom random generator. For example, rq.trio() yields one of three equally possible: -1, 0, 1.

In [21]:
#  Random trio values:
for i in range( 5 ):
    print rq.trio()

-1
1
-1
1
0


# WRAP-UP

We demonstrated the following useful functions defined in the **randquantum** module:

- seed()
- nine()
- randint()
- randpick()
- gauss()
- real()
- boolean()
- trio()

## Appendix 1: time executions

IPython has a nice way to time Python functions and expressions. Wall time will vary according to your internet speed, thus this illustrates I/O bound versus CPU bound processes.

In [24]:
%time for i in range( 10000 ): rq.nine()

CPU times: user 1.21 s, sys: 40 ms, total: 1.25 s
Wall time: 10.5 s


## Appendix 2: references and source code

*So how exactly does the randomness get introduced into a computer program?* The physical source is described below, and the data transfer relies on the function getanu(). The rest is basically a matter of mathematics.

In [26]:
# %load randquantum.py
#  Python Module for import                           Date : 2015-10-05
''' 
_______________|  randquantum.py : true random numbers using quantum mechanics. 
                      Repository : https://github.com/rsvp/randomsys

     [For simple USAGE examples, skip to the conclusion at the end.]

Reliable and unbiased random numbers are needed for a range of applications
spanning from numerical modeling to cryptographic communications. While there
are algorithms that can generate pseudo random numbers, they can never be
perfectly random nor indeterministic.

ANU researchers are generating true random numbers from a physical quantum
source by splitting a beam of light into two beams and then measuring the
power in each beam.  Because light is quantised, the light intensity in each
beam fluctuates about the mean.  Those fluctuations, due ultimately to the
quantum vacuum, can be converted into a source of random numbers. 

In classical physics, a vacuum is considered as a space that is empty of
matter or photons. In quantum mechanics, however, that same space resembles a
sea of virtual particles appearing and disappearing all the time. This result
is due to the fact that the vacuum still possesses a zero-point energy.
Consequently, the electromagnetic field of the vacuum exhibits random
fluctuations in phase and amplitude at all frequencies. By measuring these
fluctuations, one can generate ultra-high bandwidth random numbers.

The raw output of a quantum random-number generator is usually tainted by
classical technical noise. The integrity of the device can be compromised if
this noise is tampered with or even controlled by some malicious party. To
safeguard against this, our method produces side-information-independent
randomness that is quantified by min-entropy conditioned on this classical
noise. It maximizes the conditional min entropy of the number sequence
generated from a given quantum-to-classical-noise ratio. The detected
photocurrent in the experiment has a real-time random-number generation rate
of 14 (Mbit/s)/MHz. The spectral response of the detection system shows the
potential to deliver more than 70 Gbit/s of random numbers.

The 2015 hardware is constantly generating random bits at a rate of 5.7
Gbits/s.  The rate at which the live bits are streamed is limited by the
bandwidth of the internet connection.  Every number is randomly generated in
real time and cannot be predicted beforehand. 

Most pseudo random numbers have a finite period. Good pseudo random number
generators (e.g. the Mersenne Twister) have humoungous periods. But
eventually, if we wait long enough, the sequence will repeat itself. All
pseudo generators are indeed deterministic.  In contrast, for our true random
number generator, even if two exactly identical generators were placed in
identical environments with identical initial conditions, the two streams of
number generated will still be totally uncorrelated.

In development, one can use pseudo random numbers for simulations (since their
performance is not I/O bound) -- but as a final double-check on the results, 
use random numbers which are truly random.

If the server is somehow inaccessible, this module is written to fallback 
on pseudo simulation of the authentic process (with warnings emitted).


Statistical TEST RESULTS daily:  http://qrng.anu.edu.au/NIST.php

References:

- Real time demonstration of high bitrate quantum random number generation
  with coherent laser light T. Symul, S. M. Assad, and P. K. Lam:  
  Appl. Phys.  Lett. 98, 231103 (2011)

- Maximization of Extractable Randomness in a Quantum Random-Number Generator
  J. Y. Haw, S. M. Assad, A. M. Lance, N. H. Y. Ng, V. Sharma, P. K. Lam, and
  T. Symul: Phys. Rev. Applied 3, 054004 (2015)

- Australia National University, ANU:
  http://photonics.anu.edu.au/qoptics/Research/qrng.php

- General resources and papers on randomness, including statistical testing:
  http://qrng.anu.edu.au/Links.php#statistical_tests


CHANGE LOG  Latest version available at https://git.io/randomsys
2015-10-05  Introduce NINERS variable for float representation.
               Fix in-place .remove() bug in randpick by copy operation.
2015-10-03  Change real() from [0, 1.0] to [0, 1.0) for compatibility.
               Rewrite gaussquantum to use generators.
               Use Python random.randrange as offline fallback.
2015-10-02  Rename intquantum to b16quantum as a precaution.
               Add randint to supplement b16quantum.
               Rename a generator to nine. 
               Rewrite seed to use efficient generator.
               Add randpick and permute [shuffle] for arbitrarily long list.
2015-09-30  Add generator functionality for sipping stream indefinitely.
2015-09-29  First version. 
'''

import urllib2
from math   import log

from random import randrange as pseudorange  #  Only for offline fallback.
from sys    import stderr                    #  Used to warn of fallback.


NINERS = 0.99999999999
#        ^reasonable system-dependent float representation of (1 - epsilon).
#         More nines could set it exactly to 1, resulting in rare errors. 


def warn( message ):
    '''Send warning message via stderr.'''
    #  Portable for both Python 2 and 3.
    stderr.write( ' :!  Warning: ' + message + '\n')


def getanu( url='https://qrng.anu.edu.au/API/jsonI.php?length=1024&type=uint16' ):
    '''Download list of Quantum Random Numbers from Australia National University.
    See API doc: https://qrng.anu.edu.au/API/api-demo.php
    where "uint16" returns integers between 0-65535 INCLUSIVE of endpoints, 
    and maximum length permitted is 1024 (but multiple calls are permitted).

    Each time you download the "live stream" via the functions defined below,
    the server will deliver new and unique random numbers. Moreover, these
    pages are authenticated and encrypted for security.
    '''
    page = urllib2.urlopen( url, timeout=7 )
    #  print "DEBUG: retrieved json line from server."
    json = page.read()
    #  For length=3, json looks like:
    #      {"type":"uint16","length":3,"data":[7731,40732,1971],"success":true} 
    #  but we ignore the json module, and use brute force:
    json_after = json.split('[')[1]
    strlist = json_after.split(']')[0].split(',')
    #  Above read as string, so of course, convert to integer for our list:
    return [ int(s) for s in strlist ]


def randquantum_authentic( length=1000 ):
    '''Quantum random integers between [0, 65535] inclusive in a list.
    The data is online thus the performance is I/O bound.
    ''' 
    #  length of 1024 will make exactly one call to server.
    #  We must possibly make multiple calls to overcome API length limitation.
    calls = int(( length / 1024.5 ) + 1 )
    biglist = []
    for i in range( calls ):
        biglist += getanu()
    return biglist[:length]


def randquantum_pseudo( length=1000 ): 
    '''Pseudo simulation of randquantum_authentic(), intended as fallback.
    Offline call to the standard Python package random.
    '''
    return [ pseudorange(0, 65536) for i in range(length) ]


def randquantum( length=1000, authentic=True ):
    '''Authentic PRIMARY DEPENDENCY with offline FALLBACK.
    ___ATTN___  "authentic" switch for developer's debugging only.
    '''
    if authentic:
        try:
            return randquantum_authentic( length )
        except:
            warn(    "randquantum_authentic FAIL: now, PSEUDO simulation." )
            return randquantum_pseudo( length )
    else:
        warn( "authentic=False for randquantum implies PSEUDO simulation." )
        return randquantum_pseudo( length )


def boolquantum( length=1000 ):
    '''Convert randquantum to a random list of zeros and ones.
    In Python, 0 is False, anything else True, hence this is boolean.
    '''
    return [ i % 2 for i in randquantum( length ) ]


def realquantum( length=1000, endpoint=1.0 ):
    '''Convert randquantum to random real numbers: [0, endpoint]
    Discrete resolution is 1.52590219e-05 for [0,1].
    '''
    multiplier = float( endpoint ) / 65535 
    return [ i * multiplier for i in randquantum( length ) ]


def b16quantum( length=1000, endinteger=9 ):
    '''Random integers: [0, endinteger] where endinteger < 65536.
    For larger endinterger, consult randint below.
    If your endinteger is 1, we recommend boolquantum() instead.
    '''
    endpoint = endinteger + NINERS
    return [ int(r) for r in realquantum( length, endpoint ) ]



# ======================================================= GENERATORS =========== 
#  Above has focused on lists, but in practice, 
#  generators are more natural within other scripts.
#  The user should not worry about extracting an element from a random list.
#  Also these generators will cache the unused portion of a list in memory, 
#  and stand-by for a yield statement thus conserving calls to the server.
#  The generators below will "fill-up" only when needed (so this means 
#  the quantity of data is not pre-set unlike the lists above).


def sipstream( func_quantum, argtuple=(1024,) ):
     '''Generalized generator for certain randquantum functions. 
     Consider a stream to be lists being downloaded.
     This generator yields an element of a list as it is needed
     iteratively. Usage example:
          import randquantum as rq
          sip = sipstream( rq.gaussquantum, (1024, 0, 1.0) )
          #     where the tuple serves as positional arguments.
          print next( sip )
          print next( sip )
          print next( sip )
     '''
     length = argtuple[0]
     stream = apply( func_quantum, argtuple )
     i = 0
     while True:
          yield stream[i]
          i += 1
          if i == length:
               i = 0
               #   And FRESHEN the stream!
               stream = apply( func_quantum, argtuple )


# _______________ READY-MADE GENERATORS and iterating functions:
#
#  N.B. -  "Functions containing a yield statement are compiled
#           specially as generators -- when called, they return
#           a generator OBJECT that supports the iterator object
#           interface [e.g. next() or .next()]."
#  "next()" is a Python built-in for iterators since v2.6.


sip_boolean = sipstream( boolquantum,  (1024,)        )
sip_trio    = sipstream( b16quantum,   (1024, 2)      )
sip_nine    = sipstream( b16quantum,   (1024, 9)      )
sip_hundred = sipstream( b16quantum,   (1024, 100)    )
sip_real    = sipstream( realquantum,  (1024, NINERS ))
sip_cent    = sipstream( realquantum,  (1024, 100.0)  )


def boolean():  return sip_boolean.next()
def trio():     return sip_trio.next() - 1
def nine():     return sip_nine.next()
def hundred():  return sip_hundred.next()
def real():     return sip_real.next()
def cent():     return sip_cent.next()



def seed( length=19 ):
    '''Create a single random integer within given length.'''
    #  seed() turns out to be VERY useful for random integers, and nine()
    #  as a generator greatly reduces the number of calls to the server.
    s = ''
    for i in range( length):
        s += str( nine() )       
    #  Python can represent any integer up to memory limits!
    return int( s )


def randint( endinteger ):
    '''Random integer: [0, endinteger]; endinteger may be arbitrarily large!
    '''
    ilen = len( str(endinteger) )
    guess = seed( ilen )
    while guess > endinteger:
        guess = seed( ilen )
    return guess


def randpick( listing, count=1, replace=True ):
    '''Randomly pick element(s) from a list.'''
    if replace==False  and  count > len(listing):
        raise IndexError('Please adjust count <= length of listing.')    
    it = listing[:] 
    #            ^need a copy, not reference, due to in-place .remove()
    picks = []
    for k in range( count ):
        size = len( it )
        lucky = randint( size-1 )
        picks.append(  it[lucky] )
        if not replace:
            it.remove( it[lucky] )
    return picks


def shuffle( listing ):
    '''Randomly shuffle an entire list: permutation.'''
    #  Even for small len(x), the total number of permutations of x is 
    #  larger than the period of most PSEUDO random number generators; 
    #  this implies that most permutations of a long sequence can 
    #  never be generated. But this is not true for randquantum_authentic;
    #  though if the listing is huge, this could take a long time to finish.
    #
    return randpick( listing, len(listing), replace=False )


def gaussquantum( length=1000, mean=0, sdev=1.0 ):
    '''Transform random uniform to normal Gaussian distribution.

    Modified from Python random module, normalvariate function.
    Reference: Albert J. Kinderman and J.F. Monahan,
    "Computer generation of random variables using the ratio of 
    uniform deviates", ACM Trans Math Software 1977, v3:3:257-260.

    (Python's faster random.gauss() uses trig functions so we 
    guess that is the Box-Muller algorithm which we are avoiding
    since it appears to constrain abs(z) to under 7.)

    Ref: https://en.wikipedia.org/wiki/Normal_distribution
    see "Generating values" section.
    '''
    NV_MAGICCONST = 1.71552776992141
    #             = 4*exp(-0.5)/sqrt(2.0)
    gauss = []
    while len(gauss) < length:
        u1 = real()
        u2 = 1 - real()
        z = NV_MAGICCONST*(u1-0.5)/u2
        #   z is the KEY ratio essentially between
        #     two random reals both from uniform distribution.
        #     It is also the multiplier to standard deviation.
        zz = z*z/4.0
        #  Possible rejection next...
        if zz <= -log(u2):
            gauss.append( mean + (z * sdev))
            #  Approx. acceptance rate: 73% for <= condition.
    return gauss[:length]


# _______________ READY-MADE GENERATOR for standard Gaussian distribution:

sip_gauss   = sipstream( gaussquantum, (1024, 0, 1.0) )
def gauss():    return sip_gauss.next()


#  Interesting discussion regarding generating Gaussian distribution: 
#  http://stackoverflow.com/questions/75677/converting-a-uniform-distribution-to-a-normal-distribution


'''
============================================================ CONCLUSION ====== 

USAGE of these generators is very simple, for example:

     import randquantum as rq
     x = rq.gauss()
     y = rq.gauss()

So x and y are independently drawn from a standard 
Gaussian distribution, i.e. the normal N(0,1), 
and of course, they will be uncorrelated.

Illustrating random 0 or 1:

     if rq.boolean():
          print "yes, it's true."
     else:
          print "nothingness."

Such generator based functions can be invoked anywhere 
in your program without worrying about exhausting 
its list of origin. Our generators are designed to 
refresh automatically, and iterate indefinitely 
without blowing up memory space.


FOUR UNUSUAL GENERATORS:
rq.trio()    yields one of three equally possible: -1, 0, 1.
rq.nine()    yields a single integer 0 through 9    inclusive.
rq.cent()    will be real-valued between [0, 100.0] inclusive.
rq.hundred() will be an integer between  [0, 100]   inclusive.

THREE TRADITIONAL GENERATORS:
rq.boolean() yields binary-valued: 0 or 1.
rq.real() is real-valued [0,1] where both endpoints are included.
rq.gauss() is drawn from the standard normal distribution N(0,1).


FAQ:     What is the hit on performance versus pseudo random?
Answer:  Just as fast, except those milliseconds when a tiny json file
         is downloading. This is where the speed of your internet
         connection matters. You could cache in the background.


FAQ:     Why do these generators not output continuous values?
Answer:  Well, these true random numbers originate from 
         the **quantum** world where fundamentally everything
         is discrete.

Enjoy!
'''


# _______________ ALTERNATE SOURCES of randomness
#
#  - Humboldt University, Berlin: http://qrng.physik.hu-berlin.de 
#       Uses photon arrival times. Site requires registration.
#
#  - http://www.random.org was cool, but now has turned commercial.
#       Uses atmospheric noise, so it's empirical rather than 
#       theoretical in nature.


# _______________ TESTING randomness
#
#  - Start at the NIST, U.S. National Institute for Standards and Technology,
#          Computer Security Division: http://csrc.nist.gov/groups/ST/toolkit/rng
#
#  - Check for UPDATES at https://git.io/randomsys



if __name__ == "__main__":
     print "\n ::  THIS IS A MODULE for import -- not for direct execution! \n"
     raw_input('Enter something to get out: ')



 ::  THIS IS A MODULE for import -- not for direct execution! 

Enter something to get out: OK
